In [1]:
!pip install lmfit
# Import necessary libraries
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.integrate import odeint
from lmfit import minimize, Parameters, report_fit
import requests
import panel as pn
import matplotlib.pyplot as plt

pn.extension()

# Define the ODE model
def ode_model(z, t, beta, sigma, gamma):
    S, E, I, R = z
    N = S + E + I + R
    dSdt = -beta*S*I/N
    dEdt = beta*S*I/N - sigma*E
    dIdt = sigma*E - gamma*I
    dRdt = gamma*I
    return [dSdt, dEdt, dIdt, dRdt]

def ode_solver(t, initial_conditions, params):
    initE, initI, initR, initN = initial_conditions
    beta, sigma, gamma = params['beta'].value, params['sigma'].value, params['gamma'].value
    initS = initN - (initE + initI + initR)
    res = odeint(ode_model, [initS, initE, initI, initR], t, args=(beta, sigma, gamma))
    return res

# Load COVID data
response = requests.get('https://api.rootnet.in/covid19-in/stats/history')
covid_history = response.json()['data']

keys = ['day', 'total', 'confirmedCasesIndian', 'confirmedCasesForeign', 'confirmedButLocationUnidentified',
        'discharged', 'deaths']
df_covid_history = pd.DataFrame([[d.get('day'),
                                  d['summary'].get('total'),
                                  d['summary'].get('confirmedCasesIndian'),
                                  d['summary'].get('confirmedCasesForeign'),
                                  d['summary'].get('confirmedButLocationUnidentified'),
                                  d['summary'].get('discharged'),
                                  d['summary'].get('deaths')]
                                 for d in covid_history],
                    columns=keys)
df_covid_history = df_covid_history.sort_values(by='day')
df_covid_history['infected'] = df_covid_history['total'] - df_covid_history['discharged'] - df_covid_history['deaths']
df_covid_history['total_recovered_or_dead'] = df_covid_history['discharged'] + df_covid_history['deaths']

# Initialize parameters
initN = 1380000000
initE = 1000
initI = 47
initR = 0
sigma = 1/5.2
gamma = 1/2.9
R0 = 4
beta = R0 * gamma
days = 112

params = Parameters()
params.add('beta', value=beta, min=0, max=10)
params.add('sigma', value=sigma, min=0, max=10)
params.add('gamma', value=gamma, min=0, max=10)

# Define main simulation function
def main(initE, initI, initR, initN, beta, sigma, gamma, days, param_fitting):
    initial_conditions = [initE, initI, initR, initN]
    params['beta'].value, params['sigma'].value, params['gamma'].value = [beta, sigma, gamma]
    tspan = np.arange(0, days, 1)
    sol = ode_solver(tspan, initial_conditions, params)
    S, E, I, R = sol[:, 0], sol[:, 1], sol[:, 2], sol[:, 3]

    fig = go.Figure()
    if not param_fitting:
        fig.add_trace(go.Scatter(x=tspan, y=S, mode='lines+markers', name='Susceptible'))
        fig.add_trace(go.Scatter(x=tspan, y=E, mode='lines+markers', name='Exposed'))
        fig.add_trace(go.Scatter(x=tspan, y=I, mode='lines+markers', name='Infected'))
        fig.add_trace(go.Scatter(x=tspan, y=R, mode='lines+markers', name='Recovered'))
    if param_fitting:
        fig.add_trace(go.Scatter(x=tspan, y=df_covid_history.infected, mode='lines+markers', name='Infections Observed', line=dict(dash='dash')))
        fig.add_trace(go.Scatter(x=tspan, y=df_covid_history.total_recovered_or_dead, mode='lines+markers', name='Recovered/Deceased Observed', line=dict(dash='dash')))

    step = 1 if days <= 30 else 7 if days <= 90 else 30

    fig.update_layout(title='Simulation of SEIR Model',
                       xaxis_title='Day',
                       yaxis_title='Counts',
                       title_x=0.5,
                       width=400, height=300)
    fig.update_xaxes(tickangle=-90, tickformat=None, tickmode='array', tickvals=np.arange(0, days + 1, step))

    return fig

# Define parameter estimation function
def error(params, initial_conditions, tspan, data):
    sol = ode_solver(tspan, initial_conditions, params)
    return (sol[:, 2:4] - data).ravel()

def fit_parameters():
    initial_conditions = [initE, initI, initR, initN]
    beta = 1.08
    sigma = 0.02
    gamma = 0.02
    params['beta'].value = beta
    params['sigma'].value = sigma
    params['gamma'].value = gamma
    days = 45
    tspan = np.arange(0, days, 1)
    data = df_covid_history.loc[0:(days-1), ['infected', 'total_recovered_or_dead']].values

    result = minimize(error, params, args=(initial_conditions, tspan, data), method='leastsq')

    final = data + result.residual.reshape(data.shape)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=tspan, y=data[:, 0], mode='markers', name='Observed Infections', line=dict(dash='dot')))
    fig.add_trace(go.Scatter(x=tspan, y=data[:, 1], mode='markers', name='Observed Recovered/Deceased', line=dict(dash='dot')))
    fig.add_trace(go.Scatter(x=tspan, y=final[:, 0], mode='lines+markers', name='Fitted Infections'))
    fig.add_trace(go.Scatter(x=tspan, y=final[:, 1], mode='lines+markers', name='Fitted Recovered/Deceased'))
    fig.update_layout(title='Observed vs Fitted',
                       xaxis_title='Day',
                       yaxis_title='Counts',
                       title_x=0.5,
                       width=400, height=300)

    return result, fig

# Define widgets for interactive dashboard
initE_slider = pn.widgets.IntSlider(name='Initial Exposed', value=initE, start=0, end=5000, step=10)
initI_slider = pn.widgets.IntSlider(name='Initial Infected', value=initI, start=0, end=5000, step=10)
initR_slider = pn.widgets.IntSlider(name='Initial Recovered', value=initR, start=0, end=5000, step=10)
beta_slider = pn.widgets.FloatSlider(name='Beta', value=beta, start=0.0, end=10.0, step=0.1)
sigma_slider = pn.widgets.FloatSlider(name='Sigma', value=sigma, start=0.0, end=1.0, step=0.01)
gamma_slider = pn.widgets.FloatSlider(name='Gamma', value=gamma, start=0.0, end=1.0, step=0.01)
days_slider = pn.widgets.IntSlider(name='Days', value=days, start=1, end=365, step=1)
param_fitting_toggle = pn.widgets.Toggle(name='Parameter Fitting', value=False)

# Create interactive dashboard
def update_dashboard(initE, initI, initR, beta, sigma, gamma, days, param_fitting):
    if param_fitting:
        result, param_fig = fit_parameters()
        return pn.Row(
            pn.Column(pn.pane.Markdown("## Parameter Fitting"), param_fig),
            pn.pane.Markdown("### Fitted Parameters:\n" + result.params.pretty_print())
        )
    else:
        sim_fig = main(initE, initI, initR, initN, beta, sigma, gamma, days, param_fitting)
        return pn.Row(
            pn.Column(pn.pane.Markdown("## Simulation Results"), sim_fig)
        )

# Link widgets to the update function
def on_change(event):
    dashboard = update_dashboard(initE_slider.value, initI_slider.value, initR_slider.value, beta_slider.value, sigma_slider.value, gamma_slider.value, days_slider.value, param_fitting_toggle.value)
    dashboard_pane.object = dashboard

dashboard_pane = pn.pane.Markdown()
initE_slider.param.watch(on_change, 'value')
initI_slider.param.watch(on_change, 'value')
initR_slider.param.watch(on_change, 'value')
beta_slider.param.watch(on_change, 'value')
sigma_slider.param.watch(on_change, 'value')
gamma_slider.param.watch(on_change, 'value')
days_slider.param.watch(on_change, 'value')
param_fitting_toggle.param.watch(on_change, 'value')

# Layout the dashboard in a 3x3 grid
layout = pn.GridBox(
    pn.Column(pn.pane.Markdown("### Susceptible Population"), initE_slider, initI_slider, initR_slider, beta_slider, sigma_slider, gamma_slider, days_slider, param_fitting_toggle),
    pn.Column(pn.pane.Markdown("### Infected Population"), dashboard_pane),
    pn.Column(pn.pane.Markdown("### Recovered Population")),
    ncols=3,
    sizing_mode='stretch_both'
)

layout.servable()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.9/98.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00


<ipython-input-1-1ac2177b32f9>:13: UserWarning: Using Panel interactively in Colab notebooks requires the jupyter_bokeh package to be installed. Install it with:

    !pip install jupyter_bokeh

and try again.
  pn.extension()


Traceback (most recent call last):
 File "/usr/local/lib/python3.10/dist-packages/pyviz_comms/__init__.py", line 340, in _handle_msg
 self._on_msg(msg)
 File "/usr/local/lib/python3.10/dist-packages/panel/viewable.py", line 480, in _on_msg
 doc.unhold()
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/document.py", line 776, in unhold
 self.callbacks.unhold()
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 432, in unhold
 self.trigger_on_change(event)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 409, in trigger_on_change
 invoke_with_curdoc(doc, event.callback_invoker)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 444, in invoke_with_curdoc
 return f()
 File "/usr/local/lib/python3.10/dist-packages/bokeh/util/callback_manager.py", line 185, in invoke
 callback(attr, old, new)
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 478, in _comm_change
 state._handle_exception(e)
 File "/usr/local/lib/python3.10/dist-packages/panel/io/state.py", line 458, in _handle_exception
 raise exception
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 476, in _comm_change
 self._schedule_change(doc, comm)
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 458, in _schedule_change
 self._change_event(doc)
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 454, in _change_event
 self._process_events(events)
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 391, in _process_events
 self.param.update(**self_params)
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2319, in update
 restore = dict(self_._update(arg, **kwargs))
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2352, in _update
 self_._batch_call_watchers()
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2546, in _batch_call_watchers
 self_._execute_watcher(watcher, events)
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2506, in _execute_watcher
 watcher.fn(*args, **kwargs)
 File "<ipython-input-1-1ac2177b32f9>", line 156, in on_change
 dashboard_pane.object = dashboard
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 528, in _f
 instance_param.__set__(obj, val)
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 530, in _f
 return f(self, obj, val)
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 1553, in __set__
 obj.param._call_watcher(watcher, event)
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2526, in _call_watcher
 self_._execute_watcher(watcher, (event,))
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2506, in _execute_watcher
 watcher.fn(*args, **kwargs)
 File "/usr/local/lib/python3.10/dist-packages/panel/pane/base.py", line 340, in _update_pane
 self._update_object(ref, doc, root, parent, comm)
 File "/usr/local/lib/python3.10/dist-packages/panel/pane/base.py", line 258, in _update_object
 self._update(ref, old_model)
 File "/usr/local/lib/python3.10/dist-packages/panel/pane/base.py", line 520, in _update
 model.update(**self._get_properties(model.document))
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 588, in _get_properties
 properties = self._process_param_change(params)
 File "/usr/local/lib/python3.10/dist-packages/panel/pane/markup.py", line 458, in _process_param_change
 return super()._process_param_change(params)
 File "/usr/local/lib/python3.10/dist-packages/panel/pane/base.py", line 535, in _process_param_change
 params.update(self._transform_object(params.pop('object')))
 File "/usr/local/lib/python3.10/dist-packages/panel/pane/markup.py", line 431, in _transform_object
 obj = obj._repr_markdown_()
AttributeError: 'Row' object has no attribute '_repr_markdown_'

GridBox(ncols=3, sizing_mode='stretch_both')
    [0] Column
        [0] Markdown(str)
        [1] IntSlider(end=5000, name='Initial Exposed', step=10, value=1000)
        [2] IntSlider(end=5000, name='Initial Infected', step=10, value=47)
        [3] IntSlider(end=5000, name='Initial Recovered', step=10)
        [4] FloatSlider(end=10.0, name='Beta', value=1.3793103448275863)
        [5] FloatSlider(name='Sigma', step=0.01, value=0.1923076923076923)
        [6] FloatSlider(name='Gamma', step=0.01, value=0.3448275862068966)
        [7] IntSlider(end=365, name='Days', start=1, value=112)
        [8] Toggle(name='Parameter Fitting')
    [1] Column
        [0] Markdown(str)
        [1] Markdown(None)
    [2] Column
        [0] Markdown(str)

2024-11-07 15:57:32,861 ERROR: panel.reactive - Callback failed for object named 'Initial Exposed' changing property {'value': 1390} 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 391, in _process_events
    self.param.update(**self_params)
  File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2319, in update
    restore = dict(self_._update(arg, **kwargs))
  File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2352, in _update
    self_._batch_call_watchers()
  File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2546, in _batch_call_watchers
    self_._execute_watcher(watcher, events)
  File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2506, in _execute_watcher
    watcher.fn(*args, **kwargs)
  File "<ipython-input-1-1ac2177b32f9>", line 156, in on_change
    dashboard_pane.object = dashboard
  File "/usr/local/lib/python3.10/di